In [1]:
# import pandas as pd
import MagmaPandas as mp
import MagmaPandas.geochemistry.PEC as pec
import pandas as pd
import elements as e
import matplotlib.pyplot as plt
import meltInc.plotting as p

p.layout()
config = pec.PEC_configuration()
config.print()

############## Post-entrapment modification #############
#################### correction model ###################

Settings________________________________________________
Stepsize equilibration (moles)......................0.002
Stepsize crystallisation (moles)....................0.005
Decrease factor.....................................5    
FeO convergence (wt. %).............................0.05 
Kd convergence......................................0.001
Temperature convergence (°).........................0.1  


In [2]:
melt = mp.read_melt_inclusion('./melt.csv', index_col=['name'], total_col='total')
olivine = mp.read_olivine('./olivine.csv', index_col=['name'], total_col="total")

In [3]:
forsterites = olivine.forsterite
inclusions = melt.iloc[[0,1], :].copy()
ol_host = olivine.iloc[[0,1], :].copy()
Fo = 0.9
FeO_initial = 11.2
P_bar = 2e3

In [ ]:
equilibrated_inclusions = mp.MagmaFrame(index=inclusions.index, columns=inclusions.columns, dtype=float, units="wt. %", datatype="oxide")
ol_equilibrated = pd.Series(index=inclusions.index, dtype=float)

for (idx, mi), (_, ol) in zip(inclusions.iterrows(), ol_host.iterrows()):

    x, oc, kd = pec.Fe_equilibrate(mi, ol, P_bar)
    equilibrated_inclusions.loc[idx] = x.iloc[-1]
    ol_equilibrated.loc[idx] = oc

In [ ]:
equilibrated_inclusions

In [ ]:
ol_equilibrated

In [ ]:
corrected_inclusions = mp.Melt(index=inclusions.index, columns=inclusions.columns, dtype=float, units="wt. %", datatype="oxide")
ol_crystallised = pd.Series(index=inclusions.index, dtype=float)

for (idx, mi), (_, ol) in zip(equilibrated_inclusions.iterrows(), ol_host.iterrows()):

    x2, oc2, kd2, t = pec.crystallisation_correction(mi, ol, FeO_initial, P_bar)
    corrected_inclusions.loc[idx] = x2.iloc[-1]
    ol_crystallised.loc[idx] = oc2

In [ ]:
ol_crystallised

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

plt.plot(x["MgO"], x["FeO"], "-o")
plt.plot(x2["MgO"], x2["FeO"], "-D")


In [4]:
t = pec.PEC_olivine(inclusions, ol_host, P_bar, FeO_initial)

In [5]:
t.Fe_equilibrate(inplace=True)

T correction 09

In [ ]:
om